# Swap LRNA
Agent $\alpha$ swaps asset $i$ with the protocol for LRNA.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import inspect
from model.amm.omnipool_amm import swap_lrna, swap_lrna_delta_Qi, swap_lrna_delta_Ri

## Constraints

The swap will keep the swap invariant for asset $i$, $R_iQ_i$, constant.

## Requirements

#### Case 1: Asset $i$ sold
$$
\Delta r^\alpha \leq r_i^\alpha
$$
#### Case 2: LRNA sold
$$
-\Delta q^\alpha \leq q^\alpha
$$

## Updating AMM state

### Case 1: LRNA sold, $\Delta q^\alpha < 0$ specified

If $-\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

$$
\begin{align}
\Delta Q_i &= -\Delta q^\alpha\\
\Delta R_i &= R_i\frac{- \Delta Q_i}{Q_i + \Delta Q_i}(1 - f_A)\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i + L}{Q_i + \Delta Q_i}\right)\\
\Delta r_i^\alpha &= - \Delta R_i\\
\end{align}
$$

In [2]:
print(inspect.getsource(swap_lrna_delta_Ri))

def swap_lrna_delta_Ri(state: OmnipoolState, delta_qi: float, i: str) -> float:
    return state.liquidity[i] * (- delta_qi / (state.lrna[i] + delta_qi))



### Case 2: Asset $i$ bought, $\Delta r_i^\alpha > 0$ specified

$$
\begin{align}
\Delta R_i &= -\Delta r_i^\alpha\\
\Delta Q_i &= Q_i\frac{-\Delta R_i}{R_i(1 - f_A) + \Delta R_i}\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i + L}{Q_i + \Delta Q_i}\right)\\
\Delta q^\alpha &= - \Delta Q_i\\
\end{align}
$$

If $-\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

In [3]:
print(inspect.getsource(swap_lrna_delta_Qi))

def swap_lrna_delta_Qi(state: OmnipoolState, delta_ri: float, i: str) -> float:
    return state.lrna[i] * (- delta_ri / (state.liquidity[i] + delta_ri))



In [4]:
print(inspect.getsource(swap_lrna))

def swap_lrna(
        old_state: OmnipoolState,
        old_agent: Agent,
        delta_ra: float = 0,
        delta_qa: float = 0,
        tkn: str = ''
) -> tuple[OmnipoolState, Agent]:
    """Compute new state after LRNA swap"""

    new_state = old_state.copy()
    new_agent = old_agent.copy()

    if delta_qa < 0:
        delta_Q = -delta_qa
        delta_R = old_state.liquidity[tkn] * -delta_Q / (delta_Q + old_state.lrna[tkn]) * (1 - old_state.asset_fee)
        delta_L = -delta_Q * (1 + (1 - old_state.asset_fee) * old_state.lrna[tkn] / (old_state.lrna[tkn] + delta_Q))
        delta_ra = -delta_R
    elif delta_ra > 0:
        delta_R = -delta_ra
        delta_Q = old_state.lrna[tkn] * -delta_R / (old_state.liquidity[tkn] * (1 - old_state.asset_fee) + delta_R)
        delta_L = -delta_Q * (1 + (1 - old_state.asset_fee) * old_state.lrna[tkn] / (old_state.lrna[tkn] + delta_Q))
        delta_qa = -delta_Q
    else:
        # print(f'Invalid swap (delta_Qa {delta_Qa}, delta_Ra {delt

### Property test for Imbalance update
Note that we use $\gtrapprox$ and $\lessapprox$ as shorthand to indicate that the equality must *approximately* hold up to some tolerance, while the inequality must *exactly* hold.

As always, it is preferable upon implementation of these property tests to rearrange so that no division is necessary, to avoid rounding errors *in the tests themselves*.

We require that
$$
\frac{Q_i + L}{R_i} \gtrapprox \frac{Q_i^+ + L^+}{R_i^+}
$$